In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
#import torchviz
import time
import copy

import matplotlib.pyplot as plt

import ase.io

from utils.soap import compute_spherical_expansion_librascal

from utils import models

torch.set_default_dtype(torch.float64)

In [ ]:
torch.cuda.is_available()

In [ ]:
n_frames = 1000

frames = ase.io.read("data/elpasolites_10590.xyz", f":{n_frames}")
#methane_frames = ase.io.read("data/methane-100.xyz", f":{n_frames}")
energies = torch.tensor(np.loadtxt("data/elpasolites_10590_evpa.dat")[:n_frames])

n_train = int(0.8 * len(frames))

train_frames = frames[:n_train]
test_frames = frames[n_train:]

train_energies = energies[:n_train]
test_energies = energies[n_train:]

print(f"using {n_train} training frames")

In [ ]:
global_species = set()
for frame in frames:
    global_species.update(frame.numbers)

global_species = list(map(lambda u: int(u), global_species))

HYPERS_SMALL = {
    "interaction_cutoff": 5.0,
    "max_angular": 2,
    "max_radial": 2,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
    "compute_gradients": False,
    "expansion_by_species_method": "user defined",
    "global_species": global_species,
}

HYPERS_FROM_PAPER = {
    "interaction_cutoff": 5.0,
    "max_angular": 9,
    "max_radial": 12,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
    "compute_gradients": False,
    "expansion_by_species_method": "user defined",
    "global_species": global_species,
}

# Validate utilities

In [ ]:
# import utils.gap

# def structure_sum(kernel):
#     return utils.gap.common.SumStructureKernel.apply(kernel, test_slices, train_slices)

# rand_kernel = torch.rand((len(test_slices), len(train_slices)), requires_grad=True)
# torch.autograd.gradcheck(structure_sum, rand_kernel, fast_mode=True)

In [ ]:
def evaluate_and_plot_model(model, name, file):
    predicted_energies_training_set = model(
        train_spherical_expansions, train_species, train_slices
    )

    predicted_energies_test_set = model(
        test_spherical_expansions, test_species, test_slices
    )

    loss_fn = torch.nn.MSELoss()
    train_loss = loss_fn(predicted_energies_training_set.squeeze(), train_energies)
    test_loss = loss_fn(predicted_energies_test_set.squeeze(), test_energies)

    train_loss *= 100 / train_energies.std()
    test_loss *= 100 / test_energies.std()

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].scatter(train_energies, predicted_energies_training_set.detach().numpy())
    x = np.linspace(train_energies.min(), train_energies.max(), 20)
    ax[0].plot(x, x, color='r')

    ax[0].set_title(f'Training set — loss = {train_loss:.3} %RMSE')
    ax[0].set_xlabel('DFT')
    ax[0].set_ylabel('Predicted')


    ax[1].scatter(test_energies, predicted_energies_test_set.detach().numpy())
    x = np.linspace(test_energies.min(), test_energies.max(), 20)
    ax[1].plot(x, x, color='r')

    ax[1].set_title(f'Test set — loss = {test_loss:.3} %RMSE')
    ax[1].set_xlabel('DFT')
    ax[1].set_ylabel('Predicted')

    fig.suptitle(name)
    fig.savefig(file, bbox_inches="tight")

# Combining species

In [ ]:
class AtomisticDataset(torch.utils.data.Dataset):
    def __init__(self, frames, hypers, energies):
        self.spherical_expansions = []
        for frame in frames:
            se, slices = compute_spherical_expansion_librascal([frame], hypers)
            self.spherical_expansions.append(se)
        
        self.species = [torch.tensor(frame.numbers) for frame in frames]
        self.energies = energies

    def __len__(self):
        return len(self.spherical_expansions)

    def __getitem__(self, idx):
        return self.spherical_expansions[idx], self.species[idx], self.energies[idx]

In [ ]:
def collate_data_cpu(data):
    spherical_expansion = {
        lambda_: torch.vstack([d[0][lambda_] for d in data])
        for lambda_ in data[0][0].keys()
    }

    species = torch.hstack([d[1] for d in data])
    energies = torch.vstack([d[2] for d in data])

    slices = []
    start = 0
    for d in data:
        stop = start + d[1].shape[0]
        slices.append(slice(start, stop))
        start = stop

    return spherical_expansion, species, slices, energies

def collate_data_gpu(data):
    spherical_expansion, species, slices, energies = collate_data_cpu(data)

    spherical_expansion = {
        lambda_: se.to(device='cuda') for lambda_, se in spherical_expansion.items()
    }

    return spherical_expansion, species.to(device='cuda'), slices, energies.to(device='cuda')

# Optimization loop using GPU

In [ ]:
train_dataset = AtomisticDataset(train_frames, HYPERS_FROM_PAPER, train_energies)
test_dataset = AtomisticDataset(test_frames, HYPERS_FROM_PAPER, test_energies)

In [ ]:
device = "cuda"
# device = "cpu"

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=512, 
    shuffle=True,
    collate_fn=collate_data_gpu if device == "cuda" else collate_data_cpu
)

# dataloader without batching for the initial fit
train_dataloader_no_batch_cpu = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=len(train_dataset), 
    shuffle=False,
    collate_fn=collate_data_cpu
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=1, 
    shuffle=False,
    collate_fn=collate_data_gpu if device == "cuda" else collate_data_cpu
)

In [ ]:
def loss_optimizer(predicted, actual, regularizer, weights):
    loss = torch.linalg.norm(predicted - actual) ** 2
    # regularize the loss, full dataset std
    loss += regularizer / torch.std(train_energies) * torch.linalg.norm(weights) ** 2

    # TODO alternative: batch std
    # loss += regularizer / torch.std(actual) * torch.linalg.norm(weights) ** 2
    return loss

def loss_mae(predicted, actual):
    return torch.mean(torch.abs(predicted - actual))

In [ ]:
N_PSEUDO_SPECIES = 4
UPDATE_SUPPORT_POINTS = False

mixed_species_model = models.MixedSpeciesFullLinearGapModel(
    global_species, 
    n_pseudo_species=N_PSEUDO_SPECIES, 
    lambdas=[1e-3],
    optimizable_weights=True,
    random_initial_weights=True,
    detach_support_points=True,
)

# mixed_species_model = models.MixedSpeciesFullGapModel(
#     global_species, 
#     n_pseudo_species=N_PSEUDO_SPECIES, 
#     lambdas=[1e-3],
#     zeta=1,
#     optimizable_weights=True,
#     random_initial_weights=True,
#     detach_support_points=True,
# )


In [ ]:
if mixed_species_model.optimizable_weights:
    with torch.no_grad():
        # initialize the weights to a nice value using fit
        for spherical_expansions, species, slices, energies in train_dataloader_no_batch_cpu:
            # we want to intially train the model on all frames, to ensure the
            # support points come from the full dataset. This happens on CPU, the
            # move to GPU is happening below
            assert len(slices) == len(train_frames)
            mixed_species_model.fit(spherical_expansions, species, slices, energies)


if not mixed_species_model.optimizable_weights:
    # we can not use batches if we are training with linear algebra, we need to
    # have all training frames available
    assert train_dataloader.batch_size >= len(train_frames)

mixed_species_model.to(device=device)

if mixed_species_model.optimizable_weights:
    regularizer = 2e-3
else:
    regularizer = 0

lr = 0.1
optimizer = torch.optim.AdamW(
    mixed_species_model.parameters(), 
    lr=lr, weight_decay=0.0
)

# optimizer = torch.optim.LBFGS(
#     mixed_species_model.parameters(), 
#     lr=lr,
# )

In [ ]:
all_losses = []
for epoch in range(100):
    epoch_start = time.time()

    if UPDATE_SUPPORT_POINTS:
        # to update the support points, we need to move to CPU to be able to fit
        # all training data at once in memory
        mixed_species_model.to(device="cpu")
        for spherical_expansions, species, slices, _ in train_dataloader_no_batch_cpu:
            assert len(slices) == len(train_frames)
            # use `select_again=True` to re-select the same number of support
            # points. this might make convergence slower, but maybe able to
            # reach a lower final loss?
            #
            # with `select_again=False`, the environments selected in the first
            # fit above are used as support points
            mixed_species_model.update_support_points(
                spherical_expansions, species, slices, select_again=False
            )
        mixed_species_model.to(device=device)

    for spherical_expansions, species, slices, energies in train_dataloader:
        def single_step():
            optimizer.zero_grad()
           
            if not mixed_species_model.optimizable_weights:
                mixed_species_model.fit(spherical_expansions, species, slices, energies)
                
            predicted = mixed_species_model(spherical_expansions, species, slices)    

            loss = loss_optimizer(predicted, energies, regularizer, mixed_species_model.model.weights)
            loss.backward()

            return loss

        loss = optimizer.step(single_step)
        all_losses.append(loss.item())

    epoch_time = time.time() - epoch_start
    if epoch % 1 == 0:
        with torch.no_grad():
            predicted = []
            for spherical_expansions, species, slices, energies in test_dataloader:
                predicted.append(mixed_species_model(spherical_expansions, species, slices))

            predicted = torch.vstack(predicted)
            mae = loss_mae(predicted.cpu(), test_dataset.energies)

        print(f"epoch {epoch} took {epoch_time:.4}s, optimizer loss={loss.item():.4}, test mae={mae:.4}")

In [ ]:
plt.semilogy(all_losses)